In [24]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime as dt
import script as sc
import csv
import re
import signal
from interruptingcow import timeout
import requests
from bs4 import BeautifulSoup
import gender_guesser.detector as gender
import sys
from dateutil.relativedelta import relativedelta
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import sklearn.svm as svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import string

d = gender.Detector()


In [31]:
# only will work on stocks that IPO'D in the last 5 years

def createData(symbol1):
    symbol = symbol1.upper()
    try:
        with timeout(100,exception = KeyboardInterrupt):
            date = sc.simpleIPODate(symbol)
            print('Ipo Date:',date)
    except KeyboardInterrupt:
        print("Failed")
        exit()
    current = []
    location = ","
    page = requests.get('https://www.marketwatch.com/investing/stock/' + symbol.lower() + '/profile')
    contents = page.content
    soup = BeautifulSoup(contents,'html.parser')
    tds = soup.find_all('p')
    for i in tds:
        try:
            if 'headquartered in' in i.string.strip() and '...' not in i.string.strip():
                location = i.string.strip().split('headquartered in')[1].replace('.','')
                break
        except:
            pass
    soup = BeautifulSoup(contents,'html.parser')
    tdx = soup.find_all('div',{"class": "twowide"})
    soup = BeautifulSoup(str(tdx),'html.parser')
    tds = soup.find_all('p')
    fiscal = ''
    rev = ''
    net = ''
    sales = ''
    emp = ''
    catch =''
    sec = ''
    ind = ''
    for i in tds:
        if catch == 'fiscal':
            catch = ''
            fiscal = i.string.strip()
        elif catch == 'rev':
            catch = ''
            rev = i.string.strip()
        elif catch == 'net':
            catch = ''
            net = i.string.strip()
        elif catch == 'sales':
            catch = ''
            sales = i.string.strip()
        elif catch == 'emp':
            catch = ''
            emp = i.string.strip()
        elif catch == 'sec':
            catch = ''
            sec = i.string.strip()
        elif catch == 'ind':
            catch = ''
            ind = i.string.strip()
        if i.string.strip() == 'Fiscal Year-end':
            catch = 'fiscal'
        elif i.string.strip() == 'Revenue':
            catch = 'rev'
        elif i.string.strip() == 'Net Income':
            catch = 'net'
        elif i.string.strip() == '2017 Sales Growth':
            catch = 'sales'
        elif i.string.strip() == 'Employees':
            catch = 'emp'
        elif i.string.strip() == 'Sector':
            catch = 'sec'
        elif i.string.strip() == 'Industry':
            catch = 'ind'

            
            
    ceoYear = None
    ceoAge = None
    ceoName = " "
    page = requests.get('https://www.reuters.com/finance/stocks/company-officers/' + str(symbol) + '.A')
    contents = page.content
    soup = BeautifulSoup(contents, 'html.parser')
    trs = soup.find_all('td')
    for i in range(len(trs)):
        try:
            if 'Chief Executive Officer' in trs[i].string:
                ceoYear = trs[i-1].string
                ceoAge = trs[i-2].string
                ceoName = trs[i-3].text.strip().replace('\xa0',' ')
                break
        except TypeError:
            ceoYear = None
            ceoAge = None
            ceoName = ""
    t = str(date).split("-")
    try:
        tday = int(t[2])
        tmonth = int(t[1])
        tyear = int(t[0])
    except IndexError:
        tday = None
        tmonth = None
        tyear = None
    try:
        if int(dt.datetime(tyear,tmonth,tday).year) > int(ceoYear):
            ceoInCharge = 'Yes'
        elif int(dt.datetime(tyear,tmonth,tday).year) < int(ceoYear):
            ceoInCharge = 'No'
        else:
            ceoInCharge = 'sameYear'
    except TypeError:
        ceoInCharge = 'n/a'
            
    pYear = None
    pAge = None
    pName = " "   
    for i in range(len(trs)):
        try:
            if 'President' in trs[i].string:
                pYear = trs[i-1].string
                pAge = trs[i-2].string
                pName = trs[i-3].text.strip().replace('\xa0',' ')
                break
        except TypeError:
            pYear = None
            pAge = None
            pName = ""
    t = str(date).split("-")
    try:
        tday = int(t[2])
        tmonth = int(t[1])
        tyear = int(t[0])
    except IndexError:
        tday = None
        tmonth = None
        tyear = None
    try:
        if int(dt.datetime(tyear,tmonth,tday).year) > int(pYear):
            pInCharge = 'Yes'
        elif int(dt.datetime(tyear,tmonth,tday).year) < int(pYear):
            pInCharge = 'No'
        else:
            pInCharge = 'sameYear'
    except TypeError:
        pInCharge = 'n/a'
            
    city,state = location.split(",")
    city = city.strip()
    state = state.strip()
    usingDate = dt.datetime.strptime(date,'%Y-%m-%d')
    f = web.DataReader('SPY', 'iex', usingDate - dt.timedelta(days = 365 / 12), usingDate)
    monthTrend = (f.iloc[len(f) -1 ]['open'] - np.mean(f['open']) ) / np.std(f['open'])
    f = web.DataReader('SPY', 'iex', usingDate - dt.timedelta(days = 365 / 4), usingDate)
    threeMonthTrend = (f.iloc[len(f) -1 ]['open'] - np.mean(f['open']) ) / np.std(f['open'])
    f = web.DataReader('SPY', 'iex', usingDate - dt.timedelta(days = 365 / 2), usingDate)
    sixMonthTrend = (f.iloc[len(f) -1 ]['open'] - np.mean(f['open']) ) / np.std(f['open'])
    f = web.DataReader('SPY', 'iex', usingDate - dt.timedelta(days = 365 / 12), usingDate)
    yearTrend = (f.iloc[len(f) -1 ]['open'] - np.mean(f['open']) ) / np.std(f['open'])
    
    
    page = requests.get('https://www.marketwatch.com/investing/stock/' + symbol.lower() + '/profile')
    contents = page.content
    soup = BeautifulSoup(contents,'html.parser')
    tds = soup.find_all('p')
    for i in tds:
        try:
            if 'founded' in i.string.strip() and '...' not in i.string.strip():
                temp = i.string.strip().split('founded')[1].replace('.','')
                yearFounded = [int(s) for s in temp.split() if s.isdigit()][0]
        except AttributeError:
            yearFounded = np.nan
    
    ceoInCharge = 'Yes'
    pInCharge = 'Yes'
    if yearFounded == np.nan:
        yearDiff = np.nan
    else:
        yearDiff = int(date.split('-')[0]) - yearFounded
    return([symbol,city,state,fiscal,rev,net,sales,emp,ceoYear,ceoAge,ceoName,d.get_gender(ceoName.split(" ")[0]),
           ceoInCharge,pYear,pAge,pName,d.get_gender(pName.split(" ")[0]),pInCharge,sec,ind,monthTrend,
            threeMonthTrend,sixMonthTrend,yearTrend,yearFounded,yearDiff])
    


In [61]:
def predict(stock1):
#     try:
        stock = stock1.upper()
        yo = createData(stock)
        z = pd.DataFrame(np.asarray(yo).reshape(-1,len(yo)),columns = ['symbol','city','stateCountry','fiscal','rev','net','sales','employees','ceoYear','ceoAge','ceoName','CEOGender',
               'ceoInCharge','pYear','pAge','pName','PresidentGender','pInCharge','sector','ind','MarketMonthTrend','Market3MonthTrend','Market6MonthTrend','MarketYearTrend','yearFounded','yearDifference'])
        data = z.set_index('symbol')
        data['employees'] = data['employees'].str.replace(',','')
        most = []
        ua = []
        fiscalmonth = []
        cAges = []
        cGender = []
        pAges = []
        pGender = []
        yf = []
        yd = []
        for index, row in data.iterrows():
            val = row['employees']
            if val == '-':
                most.append('Blank')
            else:
                try:
                    val = int(str(val).replace(',',''))
                    if val in range(0,101):
                        most.append('0 - 100')
                    elif val in range(101,501):
                        most.append('101 - 500')
                    elif val in range(501,1001):
                        most.append('501 - 1000')
                    elif val in range(1001,5001):
                        most.append('1001 - 5000')
                    elif val in range(5001,10001):
                        most.append('5001 - 10000')
                    elif val >= 10001:
                        most.append('>10000')
                    else:
                        raise('error')
                except:
                    most.append('N/A')
            if len(str(row['stateCountry']).strip()) == 2:
                ua.append('Yes')
            elif str(row['stateCountry']).strip() == 'nan':
                ua.append('Unknown')
            else:
                ua.append('No')
            try:
                if len(str(row['fiscal'][3:6]).strip()) > 0:
                    fiscalmonth.append(row['fiscal'][3:6])
                else:
                    fiscalmonth.append('Unknown')
            except:
                fiscalmonth.append('Unknown')       
            if row['ceoInCharge'] == 'Yes' or row['ceoInCharge'] == 'sameYear':
                try:
                    age = int(row['ceoAge'])
                    if age < 30:
                        cAges.append('Under 30')
                    elif age < 40:
                        cAges.append('30 - 39')
                    elif age < 50:
                        cAges.append('40 - 49')
                    elif age < 60:
                        cAges.append('50 - 59')
                    elif age < 70:
                        cAges.append('60 - 69')
                    else:
                        cAges.append('70+')
                except:
                    cAges.append('nan')
                cGender.append(row['CEOGender'])
            elif row['ceoInCharge'] == 'No':
                cAges.append('Blank')
                cGender.append('Blank')
            else:
                cAges.append('Unknown')
                cGender.append('Unknown')
            if row['pInCharge'] == 'Yes' or row['pInCharge'] == 'sameYear':
                try:
                    age = int(row['pAge'])
                    if age < 30:
                        pAges.append('Under 30')
                    elif age < 40:
                        pAges.append('30 - 39')
                    elif age < 50:
                        pAges.append('40 - 49')
                    elif age < 60:
                        pAges.append('50 - 59')
                    elif age < 70:
                        pAges.append('60 - 69')
                    else:
                        pAges.append('70+')
                except:
                    pAges.append('nan')
                pGender.append(row['PresidentGender'])
            elif row['pInCharge'] == 'No':
                pAges.append('Blank')
                pGender.append('Blank')
            else:
                pAges.append('Unknown')
                pGender.append('Unknown')
            try:
                if int(row['yearFounded']) <= 1900:
                    yf.append('<= 1900')
                elif int(row['yearFounded']) <= 1950:
                    yf.append('1901 - 1950')
                elif int(row['yearFounded']) <= 2000:
                    yf.append('1951 - 2000')
                elif int(row['yearFounded']) <= 2010:
                    yf.append('2001 - 2010')
                elif int(row['yearFounded']) <= 2018:
                    yf.append('2011 - present')
                else:
                    yf.append('Unknown')
            except ValueError:
                yf.append(np.nan)
            try:
                val = int(row['yearDifference'])
                if val == 0:
                    yd.append('0')
                elif val == 1:
                    yd.append('1')
                elif val <=5:
                    yd.append('2 - 5')
                elif val <= 10:
                    yd.append('6 - 10')
                else:
                    yd.append('11+')
            except:
                yd.append('Unknown')


        data['employeesGrouped'] = most
        data['USACompany'] = ua
        data['FiscalMonth'] = fiscalmonth
        data['usableCEOAge'] = cAges
        data['usableCEOGender'] = cGender
        data['usablePresidentAge'] = pAges
        data['usablePresidentGender'] = pGender
        data['FoundingDateGrouped'] = yf
        data['yearDifferenceGrouped'] = yd
        train = pd.read_csv("/Users/gusmoir/Desktop/Projects/SeniorProject/finalHelper.csv",encoding = "ISO-8859-1")
        train = train.set_index('Symbol')
        trainAnswers = train[['DaysBetterThanSP','daysProfit','daysProfitGrouped','Profitable','Safe','HomeRunDay','HomeRun']]
        trainAnswers = trainAnswers[~trainAnswers.index.duplicated(keep='first')]
        data = train.append(data)
        createCategories = ['Year','Month','Day','dayOfWeek','Sector','Industry','CEOGender','PresidentGender'
                        ,'City','stateCountry','employeesGrouped','USACompany','FiscalMonth','FoundingDateGrouped'
                        ,'usableCEOAge','usableCEOGender','usablePresidentAge','usablePresidentGender','yearDifferenceGrouped']
        res1 = []
        for index,row in data.iterrows():
            res1.append(index)
        res = pd.DataFrame(res1,columns = ['Symbol'])
        res = res.set_index('Symbol')
        appl = []
        for index, row in data.iterrows():
            appl.append(index)
        for var in createCategories:
            temp = pd.get_dummies(data[var],prefix= var)
            temp['Symbol'] = appl
            temp2 = temp.set_index('Symbol')
            res = res.join(temp2)
            res = res[~res.index.duplicated(keep='first')]
        test = res.loc[stock].to_frame().transpose()
        train = res.drop(stock)
        deciders = ['Safe' ,'Profitable','HomeRun']
        print(stock)
        used = []
        for decider in deciders:
            print('   ',decider.upper() + ':',)
            testAllX = preprocessing.scale(train)
            clf = svm.SVC(cache_size = 100)
            scores = cross_val_score(clf, testAllX, trainAnswers[decider], cv=5)
            svmStats = [scores.mean(),scores.std()]
            clf.fit(testAllX,trainAnswers[decider])
            decSVM = clf.predict(test)
            grad_boost = GradientBoostingClassifier(n_estimators = 100, max_depth = 7)
            scores = cross_val_score(grad_boost, train, trainAnswers[decider], cv=5)
            grad_boostStats = [scores.mean(),scores.std()]
            grad_boost.fit(train,trainAnswers[decider])
            decGradBoost = grad_boost.predict(test)
            adaboost = AdaBoostClassifier(n_estimators = 100)
            adaboost.fit(train,trainAnswers[decider])
            scores = cross_val_score(adaboost, train, trainAnswers[decider], cv=5)
            ada_boostStats = [scores.mean(),scores.std()]
            decAdaBoost = adaboost.predict(test)
            if decider == 'Profitable':
                n = 39
            elif decider == 'Safe':
                n = 38
            elif decider == 'HomeRun':
                n = 39
            rf = RandomForestClassifier(max_depth = n, random_state = 0)
            predictedVals = cross_val_predict(rf,train,trainAnswers[decider],cv=5)
            scores = cross_val_score(rf, train, trainAnswers[decider], cv=5)
            rfStats = [scores.mean(),scores.std()]
            rf.fit(train,trainAnswers[decider])
            decRF = rf.predict(test)
            if decider == 'Profitable':
                n = 26
            elif decider == 'Safe':
                n = 16
            elif decider == 'HomeRun':
                n = 4
            neigh = KNeighborsClassifier(n_neighbors = n)
            neigh.fit(train,trainAnswers[decider])
            scores = cross_val_predict(neigh,train,trainAnswers[decider],cv=5)
            knnStats = [scores.mean(),scores.std()]
            decKNN = neigh.predict(test)


            accuracies = [svmStats[0],grad_boostStats[0],ada_boostStats[0],rfStats[0],knnStats[0]]
            decisions = [decSVM,decGradBoost,decAdaBoost,decRF,decKNN]
            use = [decSVM[0],decGradBoost[0],decAdaBoost[0],decRF[0],decKNN[0]]
            models = ['SVM','Gradient Boosting','Ada Boosting','Random Forrest','KNN']

            while len(accuracies) > 0:
                best = accuracies.index(max(accuracies))

                if decisions[best] == 1:
                    print('     ',models[best],'(',"{0:.2%}".format(round(accuracies[best],4)),')',"    Yes")
                else:
                    print('     ',models[best],'(',"{0:.2%}".format(round(accuracies[best],4)),')','    No')
                del accuracies[best]
                del decisions[best]
                del models[best]
            print()
            print()
            use.insert(0,stock1)
            use.insert(1,decider)
            used.append(use)
        return pd.DataFrame(used,columns = ['Symbol','decider','SVM','GradientBoost','AdaBoost','RandomForrest','KNN'])
#     except:
#         print("error with stock",stock1)


In [62]:
def getRecentIPOs(year,month):
    page = requests.get('https://www.nasdaq.com/markets/ipos/activity.aspx?tab=pricings&month=' + year + '-' + month)
    contents = page.content
    soup = BeautifulSoup(contents,'html.parser')
    bod = soup.find_all('tbody')
    bod1 = bod[0].find_all('a')
    potential = [x.string for x in bod1[1::2] ]
    pot = [y for y in potential if string.punctuation not in y ]
    return [y for y in pot if "'" not in y ]

In [63]:
def predictTwoMonths():
    year = dt.datetime.today().year
    month = dt.datetime.today().month
    first = getRecentIPOs(str(year),str(month))
    second = getRecentIPOs(str(year),str(month - 1))
    return first + second

In [64]:
result = pd.DataFrame([['a','a.b','b','c','d','e','f']],columns = ['Symbol','decider','SVM','GradientBoost','AdaBoost','RandomForrest','KNN'])
for x in predictTwoMonths():
    temp = predict(x)
    result = result.append([temp])

5y
Ipo Date: 2018-09-25
1y
1y
1y
1y


/Users/gusmoir/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (1342,1425,1432,1543,1546,1549,1551,1552,1553,1562,1587,1588,1605,1608,1615,1619,1620,1621,1622,1629,1630,1632,1633,1640,1641,1642,1643,1644,1646) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


VIOT
    SAFE:
      SVM ( 83.97% )     No
      Random Forrest ( 83.78% )     No
      Gradient Boosting ( 83.02% )     No
      Ada Boosting ( 82.43% )     No
      KNN ( 0.16% )     No


    PROFITABLE:
      Gradient Boosting ( 65.42% )     Yes
      Ada Boosting ( 63.07% )     Yes
      Random Forrest ( 60.94% )     No
      SVM ( 60.93% )     No
      KNN ( 51.37% )     No


    HOMERUN:
      SVM ( 92.97% )     No
      Random Forrest ( 92.97% )     No
      Gradient Boosting ( 92.46% )     No
      Ada Boosting ( 91.84% )     No
      KNN ( 0.38% )     No


5y
Ipo Date: 2018-09-21
1y
1y
1y
1y


/Users/gusmoir/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (1342,1425,1432,1543,1546,1549,1551,1552,1553,1562,1587,1588,1605,1608,1615,1619,1620,1621,1622,1629,1630,1632,1633,1640,1641,1642,1643,1644,1646) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


YMAB
    SAFE:
      SVM ( 83.97% )     No
      Random Forrest ( 83.73% )     No
      Gradient Boosting ( 82.89% )     No
      Ada Boosting ( 82.43% )     No
      KNN ( 0.16% )     No


    PROFITABLE:
      Gradient Boosting ( 65.18% )     Yes
      Ada Boosting ( 63.07% )     Yes
      Random Forrest ( 61.15% )     No
      SVM ( 60.93% )     Yes
      KNN ( 51.37% )     No


    HOMERUN:


KeyboardInterrupt: 